Imports

In [1]:
# standard modules
import copy
import os
import re
# 3rd party modules
import numpy as np
import pandas as pd
from functools import reduce
pd.DataFrame.columnstartswith = lambda self, x: [c for c in self.columns if c.startswith(x)]
pd.DataFrame.columnsmatch = lambda self, x: [c for c in self.columns if re.findall(x, c)]
# Project modules
import coimbra2023, wavelet_functions

cffi mode is CFFI_MODE.ANY
Unable to determine R home: [WinError 2] Le fichier spécifié est introuvable
R home found: C:\Program Files\R\R-4.1.3
Default options to initialize R: rpy2, --quiet, --no-save
R[write to console]: 
Attachement du package : 'dplyr'


Exception ignored from cffi callback <function _consolewrite_ex at 0x0000020A7FBE0A60>:
Traceback (most recent call last):
  File "c:\Users\phherigcoimb\Anaconda3\envs\talltower\lib\site-packages\rpy2\rinterface_lib\callbacks.py", line 133, in _consolewrite_ex
    s = conversion._cchar_to_str_with_maxlen(buf, n, _CCHAR_ENCODING)
  File "c:\Users\phherigcoimb\Anaconda3\envs\talltower\lib\site-packages\rpy2\rinterface_lib\conversion.py", line 138, in _cchar_to_str_with_maxlen
    s = ffi.string(c, maxlen).decode(encoding)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 30: invalid continuation byte
Exception ignored from cffi callback <function _consolewrite_ex at 0x0000020A7FBE0A60>:
Traceback (most recent call

configure

In [2]:
# CONFIGURE
SITES_TO_STUDY = ['FR-Fon', 'FR-Gri']

# Create setup
configure = coimbra2023.structuredData()

# Select averaging period
configure.averaging = [30, 5]

# Select output file path
configure.output_path = os.path.join(os.getcwd(), 'data/FR-Gri/CDWT/FR-Gri_CDWT{}_{}.{}mn.csv')

# Select raw input file path
# e.g.: "<PROJECT FOLDER>/eddypro_output/eddypro_raw_datasets/level_6/"
configure.raw_kwargs = {'path': os.path.join(os.getcwd(), 'data/FR-Gri/eddypro_output/eddypro_raw_datasets/level_6/')}

# Select covariances
# x*y → Cov(x, y)
# x*y*z*... → Cov(x, y)|Cov(x, z),Cov(x, ...)
configure.varstorun = ['w*co2*h2o', 'w*h2o*co2', 'w*ts*co2*h2o']

# Select period of interest
# [START_DATE, END_DATE, FILE_FREQUENCY]
configure.ymd = ['202201010030', '202201020000', '30min']

# Select wavelet method
configure.method = 'dwt'


flux processing

In [ ]:
# RUN WAVELET FLUX PROCESSING
rdw = wavelet_functions.run_wt(**vars(configure), verbosity=5)

# Merge into a single file
coimbra2023.concat_into_single_file(
    os.path.dirname(configure.output_path), 
    'FR-Gri_CDWT_full_cospectra.+.30mn.csv', 
    output_path=os.path.join('data/FR-Gri/FR-Gri_CDWT_full_cospectra_unique.30mn.csv'))


gap filling

In [ ]:
# RUN GAP FILLING
for sitename in SITES_TO_STUDY:
    data = coimbra2023.get_eddypro_output(sitename)[['TIMESTAMP', 'flag(w)', 'flag(w/co2)', 'co2_flux', 
                                                  'latitude', 'longitude', 'air_temperature', 'RH', 'u*', 'Vd']]

    data = data.merge(
        coimbra2023.get_biomet(sitename)[['TIMESTAMP', 'SW_IN']], on='TIMESTAMP', how='outer')

    previous_columns = data.columns
    data = data.merge(
        coimbra2023.get_cospectra(sitename), on='TIMESTAMP', how='left')

    for c in set(data.columns) - set(previous_columns):
        data[c] = data[c] / data.Vd

    wco2_cols = data.columnsmatch('^wco2(..wh2o)$')

    gapf_data = coimbra2023.gap_filling(data, latitude=data.latitude[0], longitude=data.longitude[0], 
        cols={'DW': {'var': 'wco2', 'flag': {'flag(w)': 2}, 'flux': wco2_cols},
              'EC': {'var': 'co2_flux', 'flag': {'flag(w)': 2, 'flag(w/co2)': 2}}})
    gapf_data.to_csv(f'data/{sitename}/{sitename}_flux_MDS_gapfilled.csv', index=False)


partitioning

In [ ]:
# RUN PARTITIONING
for sitename in SITES_TO_STUDY:
    data = coimbra2023.get_eddypro_output(sitename)[['TIMESTAMP', 'flag(w)', 'flag(w/co2)', 'co2_flux', 
                                                  'latitude', 'longitude', 'air_temperature', 'RH', 'u*', 'Vd']]

    data = data.merge(
        coimbra2023.get_biomet(sitename)[['TIMESTAMP', 'SW_IN']], on='TIMESTAMP', how='outer')
    
    coimbra2023.mkdirs('data/tmp/')
    data.to_csv(f'data/tmp/data_to_be_partitioned_{sitename}.csv', index=False)

    coimbra2023.routine_partition(sitename, os.getcwd()+'data', 
                                  os.getcwd()+f'data/tmp/data_to_be_partitioned_{sitename}.csv', 
                                  ec_name="co2_flux", dw_name="wco2")


consolidate dataset as one 

In [ ]:
all_flux_data = coimbra2023.get_eddypro_output()

all_flux_data = all_flux_data.merge(
    coimbra2023.get_biomet(), on=['co_site', 'TIMESTAMP'], how='outer', suffixes=('', '_BM'))

#all_flux_data = all_flux_data.merge(
#    coimbra2023.get_metadata(), on='co_site', how='outer', suffixes=('', '_META'))

previous_columns = all_flux_data.columns
all_flux_data = all_flux_data.merge(
    coimbra2023.get_cospectra(), on=['co_site', 'TIMESTAMP'], how='left', suffixes=('', '_DWCS'))

new_columns = set(all_flux_data.columns) - set(previous_columns)
for c in new_columns:
    all_flux_data[c] = all_flux_data[c] / all_flux_data.Vd
    all_flux_data.rename(columns={c: 'DW_' + c}, inplace=True)
del new_columns, previous_columns

all_flux_data = all_flux_data.merge(
    coimbra2023.get_gapfilling(), on=['co_site', 'TIMESTAMP'], how='left', suffixes=('', '_GAPF'))

all_flux_data = all_flux_data.merge(
    coimbra2023.get_partitioning(), on=['co_site', 'TIMESTAMP'], how='outer', suffixes=('', '_PT'))

# day or night
all_flux_data['daynight'] = (all_flux_data.TIMESTAMP.dt.hour > 9) * (all_flux_data.TIMESTAMP.dt.hour < 16)
all_flux_data.loc[np.isnan(all_flux_data.PPFD_IN)==False, 'daynight'] = (all_flux_data.daynight==False * (all_flux_data.PPFD_IN < 10))==False  

# apply partitioning
all_flux_data = coimbra2023.partitionDWCS(all_flux_data, GPP='DW_GPP', Reco='DW_Reco', NEE='DW_NEE_uStar_f', 
                                          positive='DW_wco2-+wh2o_uStar_f', negative='DW_wco2--wh2o_uStar_f')

get_dic_flux_data = lambda data=all_flux_data: {k: copy.deepcopy(data.query(f"co_site == '{k}'").reset_index(drop=True)) for k in data.co_site.unique()}
